# Implementación PointNet en Keras, Tensorflow 2.0

## Lectura y preparación del conjunto de datos

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import re
import matplotlib.pyplot as plt
import random as rand


from pathlib import Path
from sklearn.model_selection import train_test_split


### Cargar y Leer archivos _.csv_ de cada subconjunto

In [ ]:
# Linux

#path_x_train = "/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/x_train/"
#path_y_train = "/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/y_train/"
#path_x_val = "/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/x_val/"
#path_y_val = "/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/y_val/"

# MacOS

path_x_train = "/Users/felixmaral/Desktop/TFG/dataset_norm/goose_teesting/x_train/"
path_y_train = "/Users/felixmaral/Desktop/TFG/dataset_norm/goose_teesting/y_train/"
path_x_val = "/Users/felixmaral/Desktop/TFG/dataset_norm/goose_teesting/x_val/"
path_y_val = "/Users/felixmaral/Desktop/TFG/dataset_norm/goose_teesting/y_val/"

file_list_x_train = sorted(os.listdir(path_x_train))
file_list_y_train = sorted(os.listdir(path_y_train))
file_list_x_val = sorted(os.listdir(path_x_val))
file_list_y_val = sorted(os.listdir(path_y_val))

X_train = []
Y_train = []
X_val = []
Y_val = []

for file in file_list_x_train:
    df_x_train = pd.read_csv(path_x_train+file)
    X_train.append(df_x_train)

for file in file_list_y_train:
    df_y_train = pd.DataFrame()
    df_y_train['sem_label'] = pd.read_csv(path_y_train+file)
    Y_train.append(df_y_train)

for file in file_list_x_val:
    df_x_val = pd.read_csv(path_x_val+file)
    X_val.append(df_x_val)

for file in file_list_y_val:
    df_y_val = pd.DataFrame()
    df_y_val['sem_label'] = pd.read_csv(path_y_val+file)
    Y_val.append(df_y_val)

In [ ]:
# Estudio de clases

Y_train_concat = pd.concat(Y_train)
Y_val_concat = pd.concat(Y_val)
Y_concat = pd.concat(([pd.DataFrame(Y_train_concat), pd.DataFrame(Y_val_concat)]))

clases_unicas_train = Y_train_concat['sem_label'].unique()
clases_unicas_val = Y_val_concat['sem_label'].unique()
clases_unicas = set([int(x) for x in np.concatenate((clases_unicas_train, clases_unicas_val))])

print("Clases definidas en el conjunto de entrenamiento: ", clases_unicas_train)
print("\nClases definidas en el conjunto de validación: ",clases_unicas_val)
print("\nClases definidas en el conjunto de datos: ",clases_unicas)


In [ ]:
# Calcula las clases faltantes en Xtrain
clases_faltantes_train = list(set(clases_unicas_train) - set(clases_unicas_val))
clases_unicas_df = np.concatenate((clases_unicas_train, clases_faltantes_train))

# Calcula las clases existentes en el conjunto de datos
clases_unicas_df = sorted(set(clases_unicas_df))

print("En el conjunto de datos hay: ", len(clases_unicas), "clases únicas")


In [ ]:
# Histograma de frecuencia de clases

from matplotlib.ticker import FuncFormatter

# Crear el histograma
plt.figure(figsize=(16, 6))
plt.style.use('ggplot')
plt.hist(Y_concat['sem_label'], 
         bins=np.arange(Y_concat['sem_label'].min(), Y_concat['sem_label'].max() + 2), 
         edgecolor='k', 
         alpha=1, 
         align='mid')

# Chatgpt code
# Configurar el formateador para el eje Y
#ax = plt.gca()
#formatter = FuncFormatter(lambda x, _: f"{int(x):,}")  # Formato con separadores de miles
#ax.yaxis.set_major_formatter(formatter)  # Aplicar el formateador

# Personalización del gráfico
plt.title("Histograma de Etiquetas Semánticas", pad=20)
plt.xlabel("Etiqueta Semántica", labelpad=10)
plt.ylabel("Frecuencia", labelpad=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Configurar las marcas del eje X en incrementos de 1
x_ticks = np.arange(Y_concat['sem_label'].min(), Y_concat['sem_label'].max() + 2, 1)
plt.xticks(x_ticks, fontsize=8)

# Ajustar padding de los x_ticks
plt.gca().tick_params(axis='x', pad=10)  # Aumenta el espacio entre los ticks y el eje

# Mostrar el gráfico
plt.show()

In [ ]:
# Clases que faltan en el conjunto de datos

GOOSE_CLASSES = [i for i in range(0, 64)]

# Calcular clases faltantes

clases_faltantes = [i for i in GOOSE_CLASSES if i not in clases_unicas]

print(f"Hay {len(clases_faltantes)} clases que no están representadas en este conjunto de datos: ")
print(clases_faltantes)
